<a href="https://colab.research.google.com/github/IgnatInyutsin/road-damage-detection/blob/main/mytask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Конфиг

In [1]:
# подключенные дата сеты
DATASET_PATHES = ["/content/drive/MyDrive/Potholes Dataset/Dataset 1 (Simplex)/"]
# с какого изображения собирать данные
DATASET_START_INDEX  = 0
# сколько данных собирать одного типа с каждого датасета
DATASET_LIMIT = 100

EPOCHS = 50

# Вспомогательные функции

In [2]:
import cv2
import numpy as np

# нормализует изображенние
def normalize(img_path):
  # gray
  img = cv2.imread(img_path, 0)
  # downscale
  height, width = img.shape[:2]
  img = cv2.resize(img, (width//5, height//5))
  
  return img

# пара тест-значение для датасета
def generate_pair(line=[], pre_path=""):
  # собираем путь к файлу
  path = pre_path
  i = 0
  while not line[i].isnumeric():
    path += line[i] + " "
    i+=1
  path = path[:-1].replace("\\", "/").replace(".bmp", ".JPG")

  # нормализуем изображение
  try:
    img = normalize(path)
  except: 
    return None
  
  height, width = img.shape[:2]

  # маска (выходной слой)
  mask = np.zeros((height, width))
  # обводим ямы на маске
  i += 1
  while i < len(line):
    '''
    В файле каждому прямоугольнику сопоставлено 4 значения:
    1, 2 - координаты верхнего левого угла
    3 - длина стороны, сонаправлений с осью x
    4 - длина стороны, сонаправленной с осью y
    '''
    cv2.rectangle(mask, (int(line[i])//5, int(line[i+1])//5), 
                  ((int(line[i])+int(line[i+2]))//5, (int(line[i+1])+int(line[i+3]))//5),
                  (255, 255, 255),
                  -1)
    i += 4

    return (img, mask,)

# Парсинг датасета

In [ ]:
from glob import glob
import os
import cv2

x_train, y_train = [], []

for path in DATASET_PATHES:
  '''
  k - ограничитель количество итераций по файлам
  '''
  k = 0

  # собираем позитивные данные
  os.chdir(path)
  with open("simpleTrainFullPhotosSortedFullAnnotations.txt") as file:
    line = file.readline().split()

    while k < DATASET_START_INDEX:
      # пропускаем значения то индекса
      k += 1
      line = file.readline().split()

    # сами итерации
    while line and k < DATASET_LIMIT + DATASET_START_INDEX:
      k += 1

      try:
        x, y = generate_pair(line=line, pre_path=path)
      except: 
        continue
      
      x_train.append(x)
      y_train.append(y)

      line = file.readline().split()

      print(k)

  # собираем негативные данные
  k = 0
  os.chdir("Train data/Negative data")

  for file in glob("*.JPG"):
    # Начинаем собирать с заданного индекса
    if k < DATASET_START_INDEX:
      k += 1
      continue
    # Условие прерывания при превышении
    if k > 68 + DATASET_LIMIT:
      break

    img = normalize(file)
    height, width = img.shape[:2]
    x_train.append(img)
    y_train.append(np.zeros((height, width)))
    k += 1
    
    print(k)

# Модель нейросети

In [4]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, SeparableConv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, add
import keras
import os

os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

'''
Модель взята с официальной документации keras
https://keras.io/examples/vision/oxford_pets_image_segmentation/
'''

def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (1,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("sigmoid")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128]:
        x = Activation("sigmoid")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("sigmoid")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [128, 64, 32]:
        x = Activation("sigmoid")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("sigmoid")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = UpSampling2D(2)(x)

        # Project residual
        residual = UpSampling2D(2)(previous_block_activation)
        residual = Conv2D(filters, 1, padding="same")(residual)
        x = add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model

# Обучение

- Создаем модель

In [ ]:
import tensorflow as tf
import numpy as np

height, width = x_train[0].shape[:2]

# очищаем прошлую модель
keras.backend.clear_session()

model = get_model((height, width), 1)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1),
              loss="categorical_crossentropy",
              metrics=[tf.keras.metrics.Accuracy(name="accuracy", dtype=None)])

model.summary()

- Преобразуем обучающие данные в пригодный для Tensorflow формат

In [6]:
# из массива в np массив
x_train = np.array(x_train, dtype=np.int64)
y_train = np.array(y_train, dtype=np.int64)

# преобразуем в формат Tensorflow
y_train = tf.convert_to_tensor(y_train) 
x_train = tf.convert_to_tensor(x_train)

- Обучаем модель

In [ ]:
model.fit(x=x_train, y=y_train, epochs=EPOCHS, shuffle=True, batch_size=8)

- Сохраняем модель

In [ ]:
model.save("roaddamage_model")